In [2]:
pip install plyer

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#Importando as bibliotecas
import requests
import pandas as pd
from plyer import notification
import sqlite3

In [6]:
#Definindo uma função que notifica o alerta
def alerta():
    notification.notify(
            title= 'ERRO AO CONSULTAR A API',
            message='A API não está retornando um valor para acesso.',
            app_name='Python',
            timeout=10)
    return None

In [7]:
#Declara a variável que recebe a URL e seu STATUS
url = 'https://pokeapi.co/api/v2/pokemon?offset=0&limit=151'
response = requests.get(url)
data_poke = response.json()

In [8]:
#DATAFRAME df_pokemon

#Função que faz o get do ID do pokémon dentro do arquivo JSON
def get_id(url):
    response = requests.get(url)
    datapoke = response.json()
    #Se o status for 200 pega o ID
    if response.status_code == 200:
        return datapoke.get('id')
    #Senão gera o alerta
    else:
        alerta()
        return None

#Realiza o if que compara o status_code
if response.status_code == 200:
    #Declara outra variável, que recebe apenas a lista de results do JSON
    poke_results = data_poke['results']

    #Declara um DATAFRAME que recebe o valor de poke_results
    df = pd.DataFrame(poke_results)

    #Adiciona a coluna ID no DATAFRAME df_pokemon pegando o valor direto da função pelo .apply
    df['id_pokemon'] = df['url'].apply(get_id)

    #Declarando o DATAFRAME final e mostrando-o
    df_pokemon = df[['id_pokemon', 'name']]
    display(df_pokemon)

#Gera o alerta
else:
    alerta()

,id_pokemon,name
0,1,bulbasaur
1,2,ivysaur
2,3,venusaur
3,4,charmander
4,5,charmeleon
...,...,...
146,147,dratini
147,148,dragonair
148,149,dragonite
149,150,mewtwo


In [9]:
#Tratamento ".info()" para verificar o DataFrame
df_pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_pokemon  151 non-null    int64 
 1   name        151 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [10]:
#Utilizando o Tratamento "fillna" para substituir todos os dados "None" por vazio.
df_pokemon.fillna('')

,id_pokemon,name
0,1,bulbasaur
1,2,ivysaur
2,3,venusaur
3,4,charmander
4,5,charmeleon
...,...,...
146,147,dratini
147,148,dragonair
148,149,dragonite
149,150,mewtwo


In [11]:
#DATAFRAME df_poke_info

#Função que faz o get das infos do pokemon dentro do arquivo JSON
def get_info(url):
    response = requests.get(url)
    datapoke = response.json()
    #Se o status for 200 pega a base_experience
    if response.status_code == 200:
        id_pokemon = datapoke.get('id')
        base_experience = datapoke.get('base_experience')
        height = datapoke.get('height')
        weight = datapoke.get('weight')
        #Retorna todas as colunas que serão criadas no dataframe
        return id_pokemon, base_experience, height, weight
    #Senão gera o alerta
    else:
        alerta()
        return None, None, None, None


#Realiza o if que compara o status_code
if response.status_code == 200:
    #Declara outra variável, que recebe apenas a lista de results do JSON
    poke_results = data_poke['results']

    #Declara o DATAFRAME que recebe o valor de poke_results e declara o DATAFRAME FINAL
    df = pd.DataFrame(poke_results)
    df_poke_info_geral = pd.DataFrame()

    #Adicionando das colunas e dados referentes a id, base_experience, altura e peso
    df_poke_info_geral[['id_pokemon', 'base_experience', 'height', 'weight']] = df['url'].apply(get_info).apply(pd.Series)

    #Mostra os dados do DATAFRAME
    display(df_poke_info_geral)
#Gera o alerta
else:
    alerta()

,id_pokemon,base_experience,height,weight
0,1,64,7,69
1,2,142,10,130
2,3,263,20,1000
3,4,62,6,85
4,5,142,11,190
...,...,...,...,...
146,147,60,18,33
147,148,147,40,165
148,149,300,22,2100
149,150,340,20,1220


In [12]:
#Tratamento ".info()" para verificar o DataFrame
df_poke_info_geral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id_pokemon       151 non-null    int64
 1   base_experience  151 non-null    int64
 2   height           151 non-null    int64
 3   weight           151 non-null    int64
dtypes: int64(4)
memory usage: 4.8 KB


In [13]:
#Utilizando o Tratamento "fillna" para substituir todos os dados "None" por vazio.
df_poke_info_geral.fillna('')

,id_pokemon,base_experience,height,weight
0,1,64,7,69
1,2,142,10,130
2,3,263,20,1000
3,4,62,6,85
4,5,142,11,190
...,...,...,...,...
146,147,60,18,33
147,148,147,40,165
148,149,300,22,2100
149,150,340,20,1220


In [14]:
#DATAFRAME DF_POKE_STATS

#Função que faz o GET do HP, ATAQUE, DEFESA, ESPECIAL DEFESA, ESPECIAL ATAQUE e VELOCIDADE
def get_stats(url):
    response = requests.get(url)
    datapoke = response.json()
    if response.status_code == 200:
        #Declara variáveis que vão receber o próximo valor da lista de dicionarios percorrida pelo for no qual o atributo for igual ao desejado, 
        #exemplo pega o base_stat de hp quando o name for hp
        hp = next(valor_hp['base_stat'] for valor_hp in datapoke['stats'] if valor_hp['stat']['name'] == 'hp')
        ataque = next(valor_ataque['base_stat'] for valor_ataque in datapoke['stats'] if valor_ataque['stat']['name'] == 'attack')
        defesa = next(valor_defesa['base_stat'] for valor_defesa in datapoke['stats'] if valor_defesa['stat']['name'] == 'defense')
        sp_ataque = next(valor_sp_ata['base_stat'] for valor_sp_ata in datapoke['stats'] if valor_sp_ata['stat']['name'] == 'special-attack')
        sp_defesa = next(valor_sp_def['base_stat'] for valor_sp_def in datapoke['stats'] if valor_sp_def['stat']['name'] == 'special-defense')
        velocidade = next(valor_speed['base_stat'] for valor_speed in datapoke['stats'] if valor_speed['stat']['name'] == 'speed')
        return hp, ataque, defesa, sp_ataque, sp_defesa, velocidade
    else:
        alerta()
        return None, None, None, None, None, None
    
#Função que faz o get dos status do pokémon dentro do arquivo JSON
if response.status_code == 200:

    #Declara uma variável que recebe o JSON da resposta
    data_poke = response.json()
    #Pega o results dentro do JSON
    poke_results = data_poke['results']

    #Declara dois dataframes que vão receber os dados
    df = pd.DataFrame(poke_results)
    df_poke_stats = pd.DataFrame()

    #Aplica os dados no dataframe final
    df_poke_stats['id_pokemon'] = df['url'].apply(get_id)
    #Cria as colunas e aplica os valores, em seguida utiliza o pd.Series para dividir em cada um tendo uma coluna própria em vez de concatenar o resultado
    df_poke_stats[['hp', 'ataque', 'defesa', 'ataque_especial', 'defesa_especial', 'velocidade']] = df['url'].apply(get_stats).apply(pd.Series)

    display(df_poke_stats)
#Gera o alerta
else:
    alerta()

,id_pokemon,hp,ataque,defesa,ataque_especial,defesa_especial,velocidade
0,1,45,49,49,65,65,45
1,2,60,62,63,80,80,60
2,3,80,82,83,100,100,80
3,4,39,52,43,60,50,65
4,5,58,64,58,80,65,80
...,...,...,...,...,...,...,...
146,147,41,64,45,50,50,50
147,148,61,84,65,70,70,70
148,149,91,134,95,100,100,80
149,150,106,110,90,154,90,130


In [15]:
#Tratamento ".info()" para verificar o DataFrame
df_poke_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id_pokemon       151 non-null    int64
 1   hp               151 non-null    int64
 2   ataque           151 non-null    int64
 3   defesa           151 non-null    int64
 4   ataque_especial  151 non-null    int64
 5   defesa_especial  151 non-null    int64
 6   velocidade       151 non-null    int64
dtypes: int64(7)
memory usage: 8.4 KB


In [16]:
#Utilizando o Tratamento "fillna" para substituir todos os dados "None" por vazio.
df_poke_stats.fillna('')

,id_pokemon,hp,ataque,defesa,ataque_especial,defesa_especial,velocidade
0,1,45,49,49,65,65,45
1,2,60,62,63,80,80,60
2,3,80,82,83,100,100,80
3,4,39,52,43,60,50,65
4,5,58,64,58,80,65,80
...,...,...,...,...,...,...,...
146,147,41,64,45,50,50,50
147,148,61,84,65,70,70,70
148,149,91,134,95,100,100,80
149,150,106,110,90,154,90,130


In [17]:
#DATAFRAME df_poke_type

#Função que faz o get do TYPE do pokemon dentro do arquivo JSON
def get_types(url):
  response = requests.get(url)
  datapoke = response.json()
  if response.status_code == 200:
      #Realiza o try except, o comando try vai rodar o código abaixo, se o next rode e encontra um valor, ele retorna o return, caso contrário, ele ativará o except, 
      #no qual é responsável por atribuir o valor "None" na "cédula"
      try:
        #Faz a mesma consulta do get_stats, declara o nome valor_tipo que pegando o valor next de "type>name", 
        #sendo utilizado para passar no for dentro da lista "types" filtrando pelo slot indicado
        tipo1 = next(valor_tipo1['type']['name'] for valor_tipo1 in datapoke['types'] if valor_tipo1['slot'] == 1)
        tipo2 = next(valor_tipo2['type']['name'] for valor_tipo2 in datapoke['types'] if valor_tipo2['slot'] == 2)
      except StopIteration:
        #Atribui "None" ao tipo2 caso não encontre um valor no next
        tipo2 = None
      return tipo1, tipo2
  #Senão gera o alerta
  else:
      alerta()
      return None
  
#Função que faz o get dos tipos do pokémon dentro do arquivo JSON
if response.status_code == 200:
  data_poke = response.json()
  #Filtra a lista de results da API principal
  poke_results = data_poke['results']

  #Declara um DATAFRAME que recebe o poke_results
  df = pd.DataFrame(poke_results)
  #Declara um novo DATAFRAME
  df_poke_type = pd.DataFrame()

  #Inclui as colunas e os valores dentro de seus respectivos DATAFRAMES
  df_poke_type['id_pokemon'] = df['url'].apply(get_id)
  df_poke_type[['tipo_primario', 'tipo_secundario']] = df['url'].apply(get_types).apply(pd.Series)

  display(df_poke_type)
#Gera o alerta
else:
   alerta()

,id_pokemon,tipo_primario,tipo_secundario
0,1,grass,poison
1,2,grass,poison
2,3,grass,poison
3,4,fire,None
4,5,fire,None
...,...,...,...
146,147,dragon,None
147,148,dragon,None
148,149,dragon,flying
149,150,psychic,None


In [18]:
#Tratamento ".info()" para verificar o DataFrame
df_poke_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_pokemon       151 non-null    int64 
 1   tipo_primario    151 non-null    object
 2   tipo_secundario  67 non-null     object
dtypes: int64(1), object(2)
memory usage: 3.7+ KB


In [19]:
#Tratamento "isna()" para ver se possui algum dado "None"
df_poke_type.isna()

,id_pokemon,tipo_primario,tipo_secundario
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,True
4,False,False,True
...,...,...,...
146,False,False,True
147,False,False,True
148,False,False,False
149,False,False,True


In [20]:
#Utilizando o Tratamento "fillna" para substituir todos os dados "None" por vazio.
df_poke_type = df_poke_type.fillna('')
display(df_poke_type)

,id_pokemon,tipo_primario,tipo_secundario
0,1,grass,poison
1,2,grass,poison
2,3,grass,poison
3,4,fire,
4,5,fire,
...,...,...,...
146,147,dragon,
147,148,dragon,
148,149,dragon,flying
149,150,psychic,


In [21]:
#Importa a bliblioteca do Banco de Dados
import sqlite3

#Cria uma função que vai retornar um dataframe com o nome de todas as tabelas do BD
def tabelas_bd():
    #conecta o BD
    conec = sqlite3.connect('projetocoder.db')
    #Query que realiza a consulta do nome das tabelas
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    #Lendo a consulta dentro da base através do pandas
    schema = pd.read_sql_query(query, conec)
    #Encerra a conexão
    conec.close()
    return(schema)

#Cria uma função que salva os dados da planilha num dataframe (df)
#e aponta o nome da planilha na variável (nome_tabela)
#para definir essa planilha como uma tabela
def salva_bd(df, nome_tabela):
   #conecta no bd
   conec = sqlite3.connect('projetocoder.db')
   #cria um df que insere os dados da planilha e nomeia ela
   df.to_sql(nome_tabela, conec, if_exists='replace', index=False)
   #encerra a conexão
   conec.close()
   return True
#OBS, essa função não trás nada em tela, apenas insere dados nela
#o df é onde os dados serão inseridos, já o nome_tabela será o nome que a tabela ficará no banco

#Cria uma função que mostra os dados da planilha quando buscado a função com o nome do DATAFRAME entre "("")"
def carrega_bd(nome_tabela):
    #conecta no bd
    conec = sqlite3.connect('projetocoder.db')
    #Query que consulta dados do Dataframe
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conec)
    #encerra a conexão
    conec.close()
    return(df)
#OBS: Essa função faz o insert da Dataframe na tabela de BD

In [22]:
#Mostra as tabelas já criadas
tabelas_bd()

,name
0,df_pokemon
1,df_poke_info_geral
2,df_poke_stats
3,df_poke_type


In [23]:
#Salva dentro do BD uma nova tabela com os dados vindos de um DATAFRAME, a primeira instrução é o DATAFRAME e a segunda é o nome da tabela
salva_bd(df_pokemon,      #Carrega dados do dataframe
         "df_pokemon")   #Nomeia a tabela df_pokemon

salva_bd(df_poke_info_geral,     #Carrega dados do dataframe
         "df_poke_info_geral")   #Nomeia a tabela df_poke_info

salva_bd(df_poke_stats,      #Carrega dados do dataframe
         "df_poke_stats")   #Nomeia a tabela df_poke_stats

salva_bd(df_poke_type,      #Carrega dados do dataframe
         "df_poke_type")   #Nomeia a tabela df_poke_type

True

In [24]:
#Mostra a atualização das tabelas no Banco
tabelas_bd()

,name
0,df_pokemon
1,df_poke_info_geral
2,df_poke_stats
3,df_poke_type


In [26]:
#Executa a função que faz o select na tabela do banco
carrega_bd("df_poke_type")

,id_pokemon,tipo_primario,tipo_secundario
0,1,grass,poison
1,2,grass,poison
2,3,grass,poison
3,4,fire,
4,5,fire,
...,...,...,...
146,147,dragon,
147,148,dragon,
148,149,dragon,flying
149,150,psychic,
